In [2]:
"""Codes that pull and link CRSP and CUMPUSTATS Data
"""

import os
import pandas as pd
import numpy as np
import datetime as dt
import wrds
import psycopg2 
import matplotlib.pyplot as plt
from dateutil.relativedelta import *
from pandas.tseries.offsets import *
from scipy import stats

pd.set_option('display.max_columns', None)

LOOKUP_FOLDER = '~/misp/lookup_tables'

In [3]:
###################
# Connect to WRDS #
###################
conn=wrds.Connection()

Enter your WRDS username [mma3]:dachxiu
Enter your password:········
WRDS recommends setting up a .pgpass file.
You can find more info here:
https://www.postgresql.org/docs/9.5/static/libpq-pgpass.html.
Loading library list...
Done


In [72]:
###################
# Compustat Block #
###################
comp = conn.raw_sql(f"""select 
                        f.gvkey, datadate, at, pstkl, txditc,
                        pstkrv, seq, pstk, ajex,
                        fyear, 
                        apdedate, pdate, fdate, c.gvkey, f.cusip as cnum,
                        c.cik, sic as sic2, sic, naics, 
                        act, at, ceq, che, dltt, gdwl, intan, lct, lt, ppegt, ppent, txdb, wcap, /* balance sheet*/
                        dp, dvt, ebit, ebitda, epsfx, epspx, gp, ib, ni, niadj, nopi, opincar, opiti, pi, revt, /* income statement*/
                        capx, aqc, dv, chech, dpc, wcapch /* cash flow statement*/
                        from comp.names as c, comp.funda as f
                        where f.gvkey=c.gvkey /*get consolidated, standardized, industrial format statements*/
                        and f.indfmt='INDL'
                        and f.datafmt='STD'
                        and f.popsrc='D'
                        and f.consol='C'
                        and fyear>='1959'
                        """)

In [73]:
comp = comp.loc[:,~comp.columns.duplicated()]

comp['datadate']=pd.to_datetime(comp['datadate']) #convert datadate to date fmt
comp['year']=comp['datadate'].dt.year

# create preferrerd stock
comp['ps']=np.where(comp['pstkrv'].isnull(), comp['pstkl'], comp['pstkrv'])
comp['ps']=np.where(comp['ps'].isnull(),comp['pstk'], comp['ps'])
comp['ps']=np.where(comp['ps'].isnull(),0,comp['ps'])

comp['txditc']=comp['txditc'].fillna(0)

# create book equity
comp['be']=comp['seq']+comp['txditc']-comp['ps']
comp['be']=np.where(comp['be']>0, comp['be'], np.nan)

# number of years in Compustat
comp=comp.sort_values(by=['gvkey','datadate'])
comp['count']=comp.groupby(['gvkey']).cumcount()

In [74]:
comp.head()

,gvkey,datadate,at,pstkl,txditc,pstkrv,seq,pstk,ajex,fyear,apdedate,pdate,fdate,cnum,cik,sic2,sic,naics,act,ceq,che,dltt,gdwl,intan,lct,lt,ppegt,ppent,txdb,wcap,dp,dvt,ebit,ebitda,epsfx,epspx,gp,ib,ni,niadj,nopi,opincar,opiti,pi,revt,capx,aqc,dv,chech,dpc,wcapch,year,ps,be,count
0,001000,1961-12-31,NaN,0.0,0.000,NaN,NaN,NaN,3.341831,1961.0,None,None,None,000032102,None,3089,3089,None,NaN,NaN,NaN,0.100,NaN,0.0,NaN,NaN,NaN,NaN,0.000,NaN,NaN,0.0,NaN,NaN,NaN,0.37,NaN,0.050,NaN,NaN,NaN,None,None,0.100,0.900,NaN,NaN,NaN,NaN,NaN,NaN,1961,0.0,NaN,0
1,001000,1962-12-31,NaN,0.0,0.000,NaN,NaN,0.0,3.341831,1962.0,None,None,None,000032102,None,3089,3089,None,NaN,0.552,NaN,0.000,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,0.040,0.0,NaN,NaN,NaN,0.66,NaN,0.120,NaN,NaN,NaN,None,None,0.210,1.600,NaN,NaN,NaN,NaN,NaN,NaN,1962,0.0,NaN,1
2,001000,1963-12-31,NaN,0.0,0.008,0.0,0.553,0.0,3.244497,1963.0,None,None,None,000032102,None,3089,3089,None,0.408,0.553,NaN,0.015,NaN,0.0,0.322,0.345,0.557,0.431,0.008,0.086,0.046,0.0,0.000,0.046,NaN,0.01,0.392,0.003,0.003,NaN,0.024,None,None,0.004,1.457,NaN,NaN,NaN,NaN,NaN,NaN,1963,0.0,0.561,2
3,001000,1964-12-31,1.416,0.0,0.020,0.0,0.607,0.0,3.089999,1964.0,None,None,None,000032102,None,3089,3089,None,0.718,0.607,0.269,0.522,NaN,0.0,0.267,0.809,0.727,0.563,0.020,0.451,0.053,0.0,0.074,0.127,NaN,0.20,0.558,0.039,0.052,NaN,0.018,None,None,0.059,2.032,NaN,NaN,NaN,NaN,NaN,NaN,1964,0.0,0.627,3
4,001000,1965-12-31,2.310,0.0,0.000,0.0,0.491,0.0,3.089999,1965.0,None,None,None,000032102,None,3089,3089,None,0.725,0.491,0.031,1.154,NaN,0.0,0.623,1.818,1.643,1.397,0.000,0.102,0.082,0.0,-0.242,-0.160,NaN,-0.95,0.346,-0.197,-0.197,NaN,0.004,None,None,-0.300,1.688,NaN,NaN,NaN,NaN,NaN,NaN,1965,0.0,0.491,4


In [9]:
%%time
###################
# CRSP Block      #
###################
# sql similar to crspmerge macro
crsp_m = conn.raw_sql(f"""
                      select a.permno, a.permco, a.date, b.ticker, b.ncusip, b.namedt, b.nameendt,
                      b.shrcd, b.exchcd, b.siccd,
                      a.ret, a.retx, a.shrout, abs(a.prc) as prc, a.cfacpr, a.cfacshr
                      from crsp.msf as a
                      left join crsp.msenames as b
                      on a.permno=b.permno
                      and b.namedt<=a.date
                      and a.date<=b.nameendt
                      where a.date between '01/01/{1959}' and '12/31/{2020}'
                      and b.exchcd between 1 and 3
                      """) 

CPU times: user 30.5 s, sys: 4.98 s, total: 35.5 s
Wall time: 1min 48s


In [10]:
# change variable format to int
crsp_m[['permco','permno','shrcd','exchcd']]=crsp_m[['permco','permno','shrcd','exchcd']].astype(int)

# Line up date to be end of month
crsp_m['date']=pd.to_datetime(crsp_m['date'])
crsp_m['jdate']=crsp_m['date']+MonthEnd(0)

# add delisting return
dlret = conn.raw_sql("""
                     select permno, dlret, dlstdt 
                     from crsp.msedelist
                     """)
dlret.permno=dlret.permno.astype(int)
dlret['dlstdt']=pd.to_datetime(dlret['dlstdt'])
dlret['jdate']=dlret['dlstdt']+MonthEnd(0)

crsp = pd.merge(crsp_m, dlret, how='left',on=['permno','jdate'])
crsp['dlret']=crsp['dlret'].fillna(0)
crsp['ret']=crsp['ret'].fillna(0)
crsp['retadj']=(1+crsp['ret'])*(1+crsp['dlret'])-1
crsp['me']=crsp['prc'].abs()*crsp['shrout'] # calculate market equity

crsp=crsp.sort_values(by=['jdate','permco','me'])

In [13]:
crsp.tail()

,permno,permco,date,ticker,ncusip,namedt,nameendt,shrcd,exchcd,siccd,ret,retx,shrout,prc,cfacpr,cfacshr,jdate,dlret,dlstdt,retadj,me
615096,19142,56978,2019-12-31,MNPR,61023L10,2019-12-19,2019-12-31,11,3,9999.0,0.0,NaN,10421.0,16.600000,1.0,1.0,2019-12-31,0.0,2019-12-31,0.0,1.729886e+05
615097,19143,56979,2019-12-31,NWGI,65136T20,2019-12-27,2019-12-31,11,3,9999.0,0.0,NaN,11742.0,3.765000,1.0,1.0,2019-12-31,0.0,2019-12-31,0.0,4.420863e+04
615098,19145,56980,2019-12-31,SPT,85209W10,2019-12-13,2019-12-31,11,3,9999.0,0.0,NaN,38080.0,16.049999,1.0,1.0,2019-12-31,0.0,2019-12-31,0.0,6.111840e+05
615100,19147,56981,2019-12-31,WKEY,97727L20,2019-12-04,2019-12-31,31,3,9999.0,0.0,NaN,5731.0,11.740000,1.0,1.0,2019-12-31,0.0,2019-12-31,0.0,6.728194e+04
615101,19148,56982,2019-12-31,XP,G9823910,2019-12-11,2019-12-31,12,3,9999.0,0.0,NaN,347643.0,38.520000,1.0,1.0,2019-12-31,0.0,2019-12-31,0.0,1.339121e+07


In [14]:
### Aggregate Market Cap ###
# sum of me across different permno belonging to same permco a given date
crsp_summe = crsp.groupby(['jdate','permco'])['me'].sum().reset_index()
# largest mktcap within a permco/date
crsp_maxme = crsp.groupby(['jdate','permco'])['me'].max().reset_index()
# join by jdate/maxme to find the permno
crsp1=pd.merge(crsp, crsp_maxme, how='inner', on=['jdate','permco','me'])
# drop me column and replace with the sum me
crsp1=crsp1.drop(['me'], axis=1)
# join with sum of me to get the correct market cap info
crsp2=pd.merge(crsp1, crsp_summe, how='inner', on=['jdate','permco'])
# sort by permno and date and also drop duplicates
crsp2=crsp2.sort_values(by=['permno','jdate']).drop_duplicates()

In [18]:
# keep December market cap
crsp2['year']=crsp2['jdate'].dt.year
crsp2['month']=crsp2['jdate'].dt.month
crsp2=crsp2.sort_values(by=['permno','date'])

In [19]:
crsp2.tail()

,permno,permco,date,ticker,ncusip,namedt,nameendt,shrcd,exchcd,siccd,ret,retx,shrout,prc,cfacpr,cfacshr,jdate,dlret,dlstdt,retadj,me,year,month
3870452,93436,53453,2019-08-30,TSLA,88160R10,2017-02-02,2019-12-31,11,3,9999.0,-0.066222,-0.066222,179127.0,225.610001,1.0,1.0,2019-08-31,0.0,NaT,-0.066222,4.041284e+07,2019,8
3875863,93436,53453,2019-09-30,TSLA,88160R10,2017-02-02,2019-12-31,11,3,9999.0,0.067639,0.067639,180000.0,240.869995,1.0,1.0,2019-09-30,0.0,NaT,0.067639,4.335660e+07,2019,9
3881265,93436,53453,2019-10-31,TSLA,88160R10,2017-02-02,2019-12-31,11,3,9999.0,0.307427,0.307427,180245.0,314.920013,1.0,1.0,2019-10-31,0.0,NaT,0.307427,5.676276e+07,2019,10
3886671,93436,53453,2019-11-29,TSLA,88160R10,2017-02-02,2019-12-31,11,3,9999.0,0.047695,0.047695,180245.0,329.940002,1.0,1.0,2019-11-30,0.0,NaT,0.047695,5.947004e+07,2019,11
3892068,93436,53453,2019-12-31,TSLA,88160R10,2017-02-02,2019-12-31,11,3,9999.0,0.267897,0.267897,180245.0,418.329987,1.0,1.0,2019-12-31,0.0,2019-12-31,0.267897,7.540189e+07,2019,12


In [50]:
_tmp_crsp = crsp2.sort_values(['permno','date'], ascending=False)\
        .set_index(['permno', 'date'])

umds = []

for m in [3, 6, 12, 24, 36, 48, 60]:
    _tmp_crsp[f'logret_{m}']=np.log(1+_tmp_crsp['retadj'])
    umd_m = _tmp_crsp.groupby(['permno'])[f'logret_{m}'].shift(1).rolling(m, min_periods=m).sum()
    umd_m = umd_m.reset_index()
    umd_m[f'cumret_{m}']=np.exp(umd_m[f'logret_{m}'])-1
    umds += [umd_m]

dfs = [df.set_index(['permno', 'date']) for df in umds]
umd = pd.concat(dfs, axis=1).reset_index()

umd['jdate']=umd['date']+MonthEnd(0)
crsp3 = crsp2.merge(umd, how='left',on=['permno','jdate'])
crsp3 = crsp3.drop(columns=['date_x', 'date_y'])

/software/python-anaconda-2020.02-el7-x86_64/lib/python3.7/site-packages/ipykernel_launcher.py:7: RuntimeWarning: divide by zero encountered in log
  import sys


In [51]:
crsp3.tail()

,permno,permco,ticker,ncusip,namedt,nameendt,shrcd,exchcd,siccd,ret,retx,shrout,prc,cfacpr,cfacshr,jdate,dlret,dlstdt,retadj,me,year,month,logret_3,cumret_3,logret_6,cumret_6,logret_12,cumret_12,logret_24,cumret_24,logret_36,cumret_36,logret_48,cumret_48,logret_60,cumret_60
3894256,93436,53453,TSLA,88160R10,2017-02-02,2019-12-31,11,3,9999.0,-0.066222,-0.066222,179127.0,225.610001,1.0,1.0,2019-08-31,0.0,NaT,-0.066222,4.041284e+07,2019,8,0.380103,0.462435,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3894257,93436,53453,TSLA,88160R10,2017-02-02,2019-12-31,11,3,9999.0,0.067639,0.067639,180000.0,240.869995,1.0,1.0,2019-09-30,0.0,NaT,0.067639,4.335660e+07,2019,9,0.552013,0.736746,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3894258,93436,53453,TSLA,88160R10,2017-02-02,2019-12-31,11,3,9999.0,0.307427,0.307427,180245.0,314.920013,1.0,1.0,2019-10-31,0.0,NaT,0.307427,5.676276e+07,2019,10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3894259,93436,53453,TSLA,88160R10,2017-02-02,2019-12-31,11,3,9999.0,0.047695,0.047695,180245.0,329.940002,1.0,1.0,2019-11-30,0.0,NaT,0.047695,5.947004e+07,2019,11,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3894260,93436,53453,TSLA,88160R10,2017-02-02,2019-12-31,11,3,9999.0,0.267897,0.267897,180245.0,418.329987,1.0,1.0,2019-12-31,0.0,2019-12-31,0.267897,7.540189e+07,2019,12,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [75]:
#######################
# CCM Block           #
#######################
ccm=conn.raw_sql("""
                  select gvkey, lpermno as permno, linktype, linkprim, 
                  linkdt, linkenddt
                  from crsp.ccmxpf_linktable
                  where substr(linktype,1,1)='L'
                  and (linkprim ='C' or linkprim='P')
                  """)

ccm['linkdt']=pd.to_datetime(ccm['linkdt'])
ccm['linkenddt']=pd.to_datetime(ccm['linkenddt'])
# if linkenddt is missing then set to today date
ccm['linkenddt']=ccm['linkenddt'].fillna(pd.to_datetime('today'))

In [76]:
ccm1=pd.merge(comp,ccm,how='left',on=['gvkey'])

In [77]:
ccm1['yearend']=ccm1['datadate']+YearEnd(0)
ccm1['jdate']=ccm1['yearend']+MonthEnd(6)

# set link date bounds
ccm2=ccm1[(ccm1['jdate']>=ccm1['linkdt'])&(ccm1['jdate']<=ccm1['linkenddt'])]

In [78]:
ccm2['permno'] = ccm2.permno.astype(int)

/software/python-anaconda-2020.02-el7-x86_64/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [79]:
ccm2 = ccm2.drop(columns=['linktype', 'linkprim', 'linkdt', 'linkenddt', 'yearend'])

In [81]:
ccm2.shape

(324377, 57)

In [82]:
ccm2.to_csv('~/scratch-midway2/misp_data/comp_1959-2019.csv')

In [65]:
crsp3.to_csv('~/scratch-midway2/misp_data/crsp_1959-2019.csv')